## Algorithmic Design Homework 3

### Exercise 1.
Implement the binary heap-based version of the Dijkstra’s algorithm.

### Solution

In the cell below we implement the heap-based version of the Dijkstra’s algorithm on the Graph data structure written in the file `Graph.py` and using the binary heap defined in `Heap.py`.  

The heap implements all the methods we saw during the lectures and some more which we found useful during the implementation of the Dijkstra's algorithm, in particular it implements a way to decrease a value without needing the index of it in the heap, obviously to accomplish this it needs more space because it needs to store a dictionary that maps a value to a specific index, the key of this mapping is defined by the user, in our case it is the name of the vertex which is unique in the graph  

Designing the graph was more complicated because it can be implemented in different ways, from a high level perspective we chosed to define the graph as a set of vertexes and a vertex as a set of edges, in the implementation however we don't use a set but a dictionary because it allows finding a specific vertex or edge of the graph in $\Theta(1)$ time on average, furthermore defining the edge as an object can help if we want to store additional information about the connection, we didn't use this feature in the code below, however we used the fact that the vertex is an object to add the members v.d, v.prev and v.importance when they were needed. Obviously this implementation requires more space than simply storing a dictionary of {source, destination} pairs, however it allows for a very high generalizability

In [37]:
from numpy import inf
from src import *
from warnings import warn
from random import randint
from typing import TypeVar, Union
from copy import deepcopy

n = TypeVar("n")


def init_sssp(G: Graph, s: n):
    """Initializes Dijkstra algorithm

    Args:
        G (Graph): Graph to initialize
        s (n): source
    
    Raises:
        RuntimeWarning: If s is not present in the graph
    """
    u = G.get_vertex(s)
    if u is None:
        message = f"The source: {s} is not present in the graph, exiting"
        warn(message, RuntimeWarning)
        return
    
    for v in G.V():
        v.d = inf
        v.pred = None
        
    u.d = 0.

    
def relax(Q: Binary_heap, u: Vertex, v: Vertex):
    """Relaxes a node

    Args:
        Q (Heap): Queue implemented by a binary heap
        u (Vertex): Vertex already relaxed
        v (Vertex): Vertex to be relaxed
    """
    w = u.get_weight(v.name)
    if v.d > u.d + w:
        v.d = u.d + w
        v.pred = u
        Q.decrease_value(v, v)

        
def dijkstra(G: Graph, s: n):
    """Implements the dijkstra algorithm for finding the single source shortest
    path from the source s

    Args:
        G (Graph): Graph to complute the distance
        s (n): Name of the source in the graph
    
    Raises:
        RuntimeWarning: If s is not present in the graph
    """
    init_sssp(G, s)
    # Initialize Q with only the source
    Q = Binary_heap([G.get_vertex(s)], total_order=lambda x, y: x.d < y.d, dict_key=lambda x: x.name)
    while not Q.is_empty():
        u = Q.remove_min()
        # Relax all the childs of v
        for v in G.get_childs(u.name):
            Q.insert(v)
            relax(Q, u, v)


# Create a simple graph for testing
G = Graph(directed=True)
G.add_edge("A", "B", 1000)
G.add_edge("A", "C", 1)
G.add_edge("C", "D", 5)
G.add_edge("D", "B", 8)
G.add_vertex("E")

dijkstra(G, "A")
[(v.name, v.d) for v in G.V()]  

[('B', 14.0), ('A', 0.0), ('C', 1.0), ('D', 6.0), ('E', inf)]

### Exercise 2.

### Solution
The implementation of the function to add the shortcuts to a graph can be found in the file `Graph.py` at line 90, while below we will implement the bidirectional version of the dijkstra algorithm

In [65]:
def dijkstra_iteration(G: Graph, Q: Binary_heap):
    u = Q.remove_min()
    for v in G.Adj(u.name):
        v = G.get_vertex(v)
        if v.importance < u.importance:
            continue
        Q.insert(v)
        relax(Q, u, v)
    
    return u.name


def get_path(G: Graph, v: n, d: n):
    v = deepcopy(v)
    path = [v]
    w = 0.
    print(v is None)
    while v!=d and v is not None:
        print(v)
        v = G.get_vertex(v)
        edge = v.pred.get_connections()[v.name]
        path.append(edge.decompose())
        w += edge.weight
        v = v.pred.name
    
    return path, w
    

def remove_duplicates(a: list) -> list:
    return [a[i] for i in range(0, len(a)) if a[i-1]!=a[i]]


def Bidirectional_dijkstra(G: Graph, s: n, d: n):
    G_down = G.T()
    G_up = G
    G_up.add_shortcuts()
    G_down.add_shortcuts()

    init_sssp(G_up, s)
    init_sssp(G_down, d)

    Q_up = Binary_heap([G_up.get_vertex(s)], total_order=lambda x, y: x.d < y.d, dict_key=lambda x: x.name)
    Q_down = Binary_heap([G_down.get_vertex(d)], total_order=lambda x, y: x.d < y.d, dict_key=lambda x: x.name)

    vertices_up = set()
    vertices_down = set()
    common_vertex = ""
    while not Q_up.is_empty() and not Q_down.is_empty():
        up = dijkstra_iteration(G_up, Q_up)
        if up in vertices_down:
            common_vertex = up
            break

        down = dijkstra_iteration(G_down, Q_down)
        if down in vertices_up:
            common_vertex = down
            break

        vertices_up.add(up)
        vertices_down.add(down)

    p1, w1 = get_path(G_up, common_vertex, s)
    p1.reverse()
    p2, w2 = get_path(G_down, common_vertex, d)
    return remove_duplicates([*p1, *p2]), w1+w2

In [68]:
for v in G.V():
    v.importance=randint(0, 10)

Bidirectional_dijkstra(G, "A", "B")

False



AttributeError: 'NoneType' object has no attribute 'pred'

In [25]:
a = [1,2,3,43]
a.reverse()
a

[43, 3, 2, 1]